In [2]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchsummary
import scipy.io as sio
from torch.utils.data import DataLoader,Dataset,TensorDataset
from torch.autograd import Variable
import numpy as np
import h5py
from sklearn.preprocessing import minmax_scale
import torch.optim as optim
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from scipy.fftpack import fft, ifft
import scipy.signal as signal
def zscore(data):
    data_mean=np.mean(data)
    data_std=np.std(data, axis=0)
    if data_std!=0:
        data=(data-data_mean)/data_std
    else:
        data=data-data_mean
    return data
def butthigh(ecg, fs):
    b1 = np.array([0.995155038209359, -1.99031007641872, 0.995155038209359])
    a1 = np.array([1, -1.99028660262621, 0.990333550211225])
    ecg_copy = np.copy(ecg)
    ecg1 = signal.filtfilt(b1, a1, ecg_copy)
    return ecg1
def hobalka(ecg1, fs, fmin, fmax):
    ecg = np.copy(ecg1)
    n = len(ecg)
    ecg_fil = fft(ecg)
    if fmin > 0:
        imin = int(fmin / (fs / n))
    else:
        imin = 1
        ecg_fil[0] = ecg_fil[0] / 2
    if fmax < fs / 2:
        imax = int(fmax / float(fs / n))
    else:
        imax = int(n / 2)
    hamwindow = np.hamming(imax - imin)
    hamsize = len(hamwindow)
    yy = np.zeros(len(ecg_fil), dtype=complex)
    istred = int((imax + imin) / 2)
    dolni = np.arange(istred-1, imax)
    ld = len(dolni)
    yy[0: ld] = np.multiply(ecg_fil[dolni - 1], hamwindow[int(np.floor(hamsize / 2)) - 1: hamsize])
    horni = np.arange(imin-1, istred-1)
    lh = len(horni)
    end = len(yy)
    yy[end - lh - 1: end - 1] = np.multiply(ecg_fil[horni], hamwindow[0: int(np.floor(hamsize / 2))])
    ecg_fil = abs(ifft(yy)) * 2
    return ecg_fil

class  conv1d_inception_block(nn.Module):

    def __init__(self, in_ch, out_ch):
        super(conv1d_inception_block, self).__init__()

        self.conv1_1 = nn.Sequential(
            nn.Conv1d(in_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=True),
            nn.Dropout(0.3))
        self.conv1_3 = nn.Sequential(
            nn.Conv1d(in_ch, out_ch, kernel_size=5, stride=1, padding=2, bias=True),
            nn.Dropout(0.3))
        self.conv1_5 = nn.Sequential(
            nn.Conv1d(in_ch, out_ch, kernel_size=7, stride=1, padding=3, bias=True),
            nn.Dropout(0.3))
        self.conv= nn.Sequential(
            nn.Conv1d(in_ch, out_ch, kernel_size=1, stride=1, padding=0, bias=True),
            #nn.Conv1d(in_ch, out_ch, kernel_size=1, stride=1, padding=0, bias=True),
            nn.Dropout(0.3),
            nn.BatchNorm1d(out_ch),
            nn.ReLU())
    def forward(self, x):

        x1 = self.conv1_1(x)
        x3 = self.conv1_3(x)
        x5 = self.conv1_5(x)
        return self.conv(x1+x3+x5)
class Recurrent_block(nn.Module):

    def __init__(self, out_ch, t=2):
        super(Recurrent_block, self).__init__()
        #self.drop_layer = nn.Dropout(0.5)
        self.t = t
        self.out_ch = out_ch
        self.conv = nn.Sequential(

            conv1d_inception_block(out_ch,out_ch),
            nn.Dropout(0.3),
            nn.BatchNorm1d(out_ch),
            nn.ReLU()
        )
        self.conv1_1 = nn.Sequential(
            nn.Conv1d(out_ch, out_ch, kernel_size=1, stride=1, padding=0, bias=True))
    def forward(self, x):
        for i in range(self.t):
            if i == 0:
                x1 = self.conv(x)
            out = self.conv1_1(x1 + x)   # out variable ta x1 howar kotha chilo na??
        return out

class Residual_block(nn. Module):

    def __init__(self, out_ch, t=2):
        super(Residual_block, self).__init__()
        #self.drop_layer = nn.Dropout(0.5)
        self.t = t
        self.out_ch = out_ch
        self.conv = nn.Sequential(

            conv1d_inception_block(out_ch,out_ch),
            nn.Dropout(0.3),
            nn.BatchNorm1d(out_ch),
            nn.ReLU()
        )
        self.conv1_1 = nn.Sequential(
            nn.Conv1d(out_ch, out_ch, kernel_size=1, stride=1, padding=0, bias=True))
    def forward(self, x):
        x1 = self.conv(x)
        x1 = self.conv(x1)
        out=self.conv1_1(x1+x)
        return out






class R_1Dcnn_RCNN_block(nn.Module):
    def __init__(self, in_ch, out_ch, t=2):
        super(R_1Dcnn_RCNN_block, self).__init__()

        self.RCNN1 = nn.Sequential(
            Recurrent_block(out_ch, t=t))

        self.RCNN2 = nn.Sequential(
            conv1d_inception_block(out_ch, out_ch))

        self.RCNN3 =nn.Sequential(
            Residual_block(out_ch, out_ch))

        self.Conv = nn.Conv1d(in_ch, out_ch, kernel_size=1, stride=1, padding=0)
        self.Conv1_1 = nn.Sequential(
            nn.Conv1d(out_ch, out_ch, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm1d(out_ch),
            nn.ReLU())
    def forward(self, x):
        x=self.Conv(x)
        x1 = self.RCNN3(x)
        x2 = self.RCNN2(x)
        x3 = self.RCNN1(x)
        out = self.Conv1_1(x3+x2+x1)
        return out
'''
class Attention_block_self(nn.Module):

    def __init__(self, F_l, F_int):
        super(Attention_block_self, self).__init__()

        self.W_g = nn.Sequential(
            nn.Conv1d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm1d(F_int)
        )

        self.psi = nn.Sequential(
            nn.Conv1d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm1d(1),
            nn.Sigmoid()
        )

        self.Tanh = nn.Tanh()

    def forward(self,  x):
        x1 = self.W_g(x)
        psi = self.Tanh(x1)
        psi = self.psi(psi)
        out = x * psi
        return out
'''

class model_1d(nn.Module):

    def __init__(self, img_ch=1, output_ch=1, t=1):
        super(model_1d, self).__init__()

        n1 =6
        filters = [n1, n1 * 2, n1 * 4, n1 * 8, n1 * 16]
        self.Maxpool0 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.Maxpool1 = nn.MaxPool1d(kernel_size=8, stride=8)
        self.Maxpool2 = nn.MaxPool1d(kernel_size=4, stride=4)
        self.RRCNN1 = R_1Dcnn_RCNN_block(img_ch, filters[3], t=t)
        self.RRCNN2 = R_1Dcnn_RCNN_block(filters[3], filters[2], t=t)
        self.RRCNN3 = R_1Dcnn_RCNN_block(filters[2], filters[2], t=t)
        self.Softmax = nn.LogSoftmax()
        self.fc1 = nn.Linear(168 ,2)
    def forward(self, x):
        x=self.Maxpool0(x)
        e1 = self.RRCNN1(x)
        # print(e1.size())
        e2 = self.Maxpool2(e1)
        # print(e2.size())
        e2 = self.RRCNN2(e2)
        e3 = self.Maxpool2(e2)
        e3 = self.RRCNN3(e3)
        e3 = self.Maxpool2(e3)
        # print(e3.size())
        e4=self.Maxpool2(e3)
        # print(e4.size())
        e7= e4.view(e4.size(0),e4.size(1)*e4.size(2))
        # print(e7.size())
        out = self.fc1(e7)
        out =self.Softmax(out)
        return out


def train(x,model,los_train, acc_train):
    optimizer = torch.optim.SGD(model.parameters(),lr=0.0050,  momentum=0.90, dampening=0, weight_decay=0.0001, nesterov=False)
    epoch=x
    j=x
    model.train()
    running_loss = 0.0
    train_correct=0.0
    for i,data in enumerate(trainloader):
        inputs, labels = data
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        optimizer.zero_grad()  # zero the gradient buffers
        output =  model(inputs)
        accracy =np.mean( (torch.argmax(output.cpu(),1)==torch.argmax(labels.cpu(),1)).numpy())
        # accracy =np.mean( (torch.argmax(output.cuda(),1)==torch.argmax(labels.cuda(),1)).numpy())
        loss = criterion(output, torch.argmax(labels, dim=1))
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        train_correct+=accracy
        if i % np.ceil(len(trainloader.dataset)/32) == np.ceil(len(trainloader.dataset)/32)-1:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f  train_acc:%.4f'%(epoch + 1, i + 1, running_loss /(len(trainloader.dataset)/32), train_correct/(len(trainloader.dataset)/32)))
            los_train[j]= running_loss /(len(trainloader.dataset)/32)
            acc_train[j]= train_correct/(len(trainloader.dataset)/32)
            running_loss = 0.0
            train_correct=0.0
    return los_train, acc_train

def validation(j,model,los_test, acc_test):
     model.eval()
     test_loss = 0.0
     test_correct=0.0
     for inputs1, labels1 in testloader:
        inputs1, labels1 = Variable(inputs1.cuda()), Variable(labels1.cuda())
        output =  model(inputs1)
        accracy1 =(torch.argmax(output.cpu(),1)==torch.argmax(labels1.cpu(),1)).numpy().sum()
        loss1 = criterion(output, torch.argmax(labels1, dim=1))
        test_loss+=loss1.item()
        test_correct+=accracy1
     test_loss /= (len(testloader.dataset)/32)
     test_correct/=(len(testloader.dataset))
     los_test[j]= test_loss
     acc_test[j]= test_correct
     print('test_loss:%.4f, test_correct%.4f'%(test_loss,test_correct))

def test():
     model.eval()
     test_loss = 0.0
     test_correct=0.0
     for inputs1, labels1 in testloader:
        inputs1, labels1 = Variable(inputs1.cuda()), Variable(labels1.cuda())
        output =  model(inputs1)
        accracy1 =(torch.argmax(output.cpu(),1)==torch.argmax(labels1.cpu(),1)).numpy().sum()
        loss1 = criterion(output, torch.argmax(labels1, dim=1))
        test_loss+=loss1.item()
        test_correct+=accracy1
     test_loss /= (len(testloader.dataset)/32)  # 32 is batch size
     test_correct/=(len(testloader.dataset))
     print('test_loss:%.4f, test_correct%.4f'%(test_loss,test_correct))


In [ ]:
# import pickle
# with open('training_dataset_2.pkl', 'rb') as f:
#      dataset = pickle.load(f)

In [3]:
data = sio.loadmat('ecgpart_02.mat')['ecgpart']
data.shape

(8218, 4000)

In [5]:
label = sio.loadmat('CSPC2020label_02_0_1_by_manurally.mat')['sqi']
label = label.reshape(8218, 1)
label.shape

(8218, 1)

In [6]:
from sklearn.model_selection import StratifiedShuffleSplit  # spliting into training and testing dataset
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
(train_index, test_index) = sss.split(data, label).__next__()

In [7]:
import pandas as pd
main_dataset = pd.DataFrame(np.concatenate((data, label), axis=1))
dataset = main_dataset.iloc[train_index].values


In [ ]:
dataset.shape

(6574, 4001)

In [ ]:
ecga = dataset[:, 0:4000]
labela = dataset[:, -1].reshape(6574, 1)

file_number=0
sfolder = StratifiedKFold(n_splits=10,random_state=1,shuffle=True)
for traindata, testdata in sfolder.split(ecga,labela):
    file_number=file_number+1
    ecgc=ecga[traindata]
    ecgt=ecga[testdata]
    labelc=labela[traindata]
    labelt=labela[testdata]
    for FF in range(len(ecgc)):
        ecgc[FF,:]=butthigh(zscore(ecgc[FF,:]),4000)
    for FF1 in range(len(ecgt)):
        ecgt[FF1,:]=butthigh(zscore(ecgt[FF1,:]),4000)

    ecgc=torch.FloatTensor(ecgc)
    ecgc=ecgc.unsqueeze(1)
    labelc=to_categorical(labelc)
    labelc=torch.FloatTensor(labelc)
    deal_dataset = TensorDataset(ecgc,labelc)
    trainloader=DataLoader(dataset=deal_dataset,batch_size=32,shuffle=True,num_workers=0)

    ecgt=torch.FloatTensor(ecgt)
    ecgt=ecgt.unsqueeze(1)
    labelt=to_categorical(labelt)
    labelt=torch.FloatTensor(labelt)
    deal_test_dataset = TensorDataset(ecgt,labelt)
    testloader=DataLoader(dataset=deal_test_dataset,batch_size=32,shuffle=True,num_workers=0)

    model=model_1d()
    model = model.cuda()
    #torchsummary.summary(model.cuda(), input_size=(1,512,241))
    criterion = nn.CrossEntropyLoss()
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.8)
    los_train=np.zeros(200)
    acc_train=np.zeros(200)
    los_test=np.zeros(200)
    acc_test=np.zeros(200)
    for x in range(200):
        los_train, acc_train=train(x,model,los_train, acc_train)
        validation(x,model,los_test, acc_test)
        if acc_test[x]>0.85:
            matname='conmodel_'+str(file_number)+'_epoch'+str(x)+'.pkl'
            torch.save(model, matname)
    filename_save='c11inception_10foldaccloss_model'+str(file_number)+'.mat'
    sio.savemat(filename_save, {'los_train':los_train,'acc_train':acc_train,'los_test':los_test,'acc_test':acc_test})

<ipython-input-1-7dd1ea3725cf>:217: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out =self.Softmax(out)


[1,   185] loss: 0.289  train_acc:0.8693
test_loss:0.2734, test_correct0.9012
[2,   185] loss: 0.136  train_acc:0.9437
test_loss:0.3489, test_correct0.8617
[3,   185] loss: 0.104  train_acc:0.9586
test_loss:0.1482, test_correct0.9605
[4,   185] loss: 0.100  train_acc:0.9635
test_loss:0.1721, test_correct0.9574
[5,   185] loss: 0.094  train_acc:0.9663
test_loss:0.1551, test_correct0.9650
[6,   185] loss: 0.092  train_acc:0.9668
test_loss:0.0973, test_correct0.9726
[7,   185] loss: 0.089  train_acc:0.9643
test_loss:0.1423, test_correct0.9650
[8,   185] loss: 0.089  train_acc:0.9697
test_loss:0.0914, test_correct0.9726
[9,   185] loss: 0.079  train_acc:0.9732
test_loss:0.0970, test_correct0.9711
[10,   185] loss: 0.082  train_acc:0.9701
test_loss:0.1115, test_correct0.9726
[11,   185] loss: 0.081  train_acc:0.9716
test_loss:0.0923, test_correct0.9726
[12,   185] loss: 0.075  train_acc:0.9738
test_loss:0.1466, test_correct0.9666
[13,   185] loss: 0.075  train_acc:0.9733
test_loss:0.1334, t

KeyboardInterrupt: ignored

In [19]:
# -*- coding: utf-8 -*-


import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchsummary
import scipy.io as sio
from torch.utils.data import DataLoader,Dataset,TensorDataset
from torch.autograd import Variable
import numpy as np
import h5py
from sklearn.preprocessing import minmax_scale
import torch.optim as optim
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from scipy.fftpack import fft, ifft
import scipy.signal as signal

# print(torch.cuda.is_available())
def zscore(data):
    data_mean=np.mean(data)
    data_std=np.std(data, axis=0)
    if data_std!=0:
        data=(data-data_mean)/data_std
    else:
        data=data-data_mean
    return data
def butthigh(ecg, fs):
    # wp = 1
    # ws = 0.5
    # Rp = 0.3
    # Rs = 2
    # [N1, Wn1] = signal.buttord(wp / (fs / 2), ws / (fs / 2), Rp, Rs)
    # [N1, Wn1] = signal.buttord(wp / (fs*1.0 / 2), ws / (fs*1.0 / 2), Rp, Rs)

    # [b1, a1] = signal.butter(N1, Wn1, 'high')
    b1 = np.array([0.995155038209359, -1.99031007641872, 0.995155038209359])
    a1 = np.array([1, -1.99028660262621, 0.990333550211225])
    ecg_copy = np.copy(ecg)
    ecg1 = signal.filtfilt(b1, a1, ecg_copy)
    return ecg1
def hobalka(ecg1, fs, fmin, fmax):
    ecg = np.copy(ecg1)
    n = len(ecg)
    ecg_fil = fft(ecg)
    if fmin > 0:
        imin = int(fmin / (fs / n))
    else:
        imin = 1
        ecg_fil[0] = ecg_fil[0] / 2
    if fmax < fs / 2:
        imax = int(fmax / float(fs / n))
    else:
        imax = int(n / 2)
    hamwindow = np.hamming(imax - imin)
    hamsize = len(hamwindow)
    yy = np.zeros(len(ecg_fil), dtype=complex)
    istred = int((imax + imin) / 2)
    dolni = np.arange(istred-1, imax)
    ld = len(dolni)
    yy[0: ld] = np.multiply(ecg_fil[dolni - 1], hamwindow[int(np.floor(hamsize / 2)) - 1: hamsize])
    horni = np.arange(imin-1, istred-1)
    lh = len(horni)
    end = len(yy)
    yy[end - lh - 1: end - 1] = np.multiply(ecg_fil[horni], hamwindow[0: int(np.floor(hamsize / 2))])
    ecg_fil = abs(ifft(yy)) * 2
    return ecg_fil
#����׼��������loader��ʽ
class  conv1d_inception_block(nn.Module):
    """
    Convolution Block 1d
    """
    def __init__(self, in_ch, out_ch):
        super(conv1d_inception_block, self).__init__()

        self.conv1_1 = nn.Sequential(
            nn.Conv1d(in_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=True),
            nn.Dropout(0.3))
        self.conv1_3 = nn.Sequential(
            nn.Conv1d(in_ch, out_ch, kernel_size=5, stride=1, padding=2, bias=True),
            nn.Dropout(0.3))
        self.conv1_5 = nn.Sequential(
            nn.Conv1d(in_ch, out_ch, kernel_size=7, stride=1, padding=3, bias=True),
            nn.Dropout(0.3))
        self.conv= nn.Sequential(
            nn.Conv1d(in_ch, out_ch, kernel_size=1, stride=1, padding=0, bias=True),
            #nn.Conv1d(in_ch, out_ch, kernel_size=1, stride=1, padding=0, bias=True),
            nn.Dropout(0.3),
            nn.BatchNorm1d(out_ch),
            nn.ReLU())
    def forward(self, x):

        x1 = self.conv1_1(x)
        x3 = self.conv1_3(x)
        x5 = self.conv1_5(x)
        return self.conv(x1+x3+x5)
class Recurrent_block(nn.Module):
    """
    Recurrent Block for R2Unet_CNN
      '''
            conv1d_inception_block(out_ch,out_ch),
            nn.Dropout(0.2),
            nn.Conv1d(out_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=True),
            nn.Dropout(0.2),
            '''
    """
    def __init__(self, out_ch, t=2):
        super(Recurrent_block, self).__init__()
        #self.drop_layer = nn.Dropout(0.5)
        self.t = t
        self.out_ch = out_ch
        self.conv = nn.Sequential(

            conv1d_inception_block(out_ch,out_ch),
            nn.Dropout(0.3),
            nn.BatchNorm1d(out_ch),
            nn.ReLU()
        )
        self.conv1_1 = nn.Sequential(
            nn.Conv1d(out_ch, out_ch, kernel_size=1, stride=1, padding=0, bias=True))
    def forward(self, x):
        for i in range(self.t):
            if i == 0:
                x1 = self.conv(x)
            out = self.conv1_1(x1 + x)
        return out

class Residual_block(nn.Module):
    """
    Recurrent Block for R2Unet_CNN
      '''
            conv1d_inception_block(out_ch,out_ch),
            nn.Dropout(0.2),
            nn.Conv1d(out_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=True),
            nn.Dropout(0.2),
            '''
    """
    def __init__(self, out_ch, t=2):
        super(Residual_block, self).__init__()
        #self.drop_layer = nn.Dropout(0.5)
        self.t = t
        self.out_ch = out_ch
        self.conv = nn.Sequential(

            conv1d_inception_block(out_ch,out_ch),
            nn.Dropout(0.3),
            nn.BatchNorm1d(out_ch),
            nn.ReLU()
        )
        self.conv1_1 = nn.Sequential(
            nn.Conv1d(out_ch, out_ch, kernel_size=1, stride=1, padding=0, bias=True))
    def forward(self, x):
        x1 = self.conv(x)
        x1 = self.conv(x1)
        out=self.conv1_1(x1+x)
        return out


class R_1Dcnn_RCNN_block(nn.Module):
    def __init__(self, in_ch, out_ch, t=2):
        super(R_1Dcnn_RCNN_block, self).__init__()

        self.RCNN1 = nn.Sequential(
            Recurrent_block(out_ch, t=t))

        self.RCNN2 = nn.Sequential(
            conv1d_inception_block(out_ch, out_ch))

        self.RCNN3 = nn.Sequential(
            Residual_block(out_ch, out_ch))

        self.Conv = nn.Conv1d(in_ch, out_ch, kernel_size=1, stride=1, padding=0)
        self.Conv1_1 = nn.Sequential(
            nn.Conv1d(out_ch, out_ch, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm1d(out_ch),
            nn.ReLU())

    def forward(self, x):
        x = self.Conv(x)
        x1 = self.RCNN3(x)
        x2 = self.RCNN2(x)
        x3 = self.RCNN1(x)
        out = self.Conv1_1(x3 + x2 + x1)
        return out




class R_inception_RCNN_block(nn.Module):
    """
    Recurrent Residual Convolutional Neural Network Block
    """
    def __init__(self, in_ch, out_ch, t=2):
        super(R_inception_RCNN_block, self).__init__()

        self.RCNN1 = nn.Sequential(
            Recurrent_block(out_ch, t=t))

        self.RCNN2 = nn.Sequential(
            conv1d_inception_block(out_ch, out_ch))

        self.RCNN3 =nn.Sequential(
            Residual_block(out_ch, out_ch))


        self.Conv = nn.Conv1d(in_ch, out_ch, kernel_size=1, stride=1, padding=0)
        self.Conv1_1 = nn.Sequential(
            nn.Conv1d(out_ch, out_ch, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm1d(out_ch),
            nn.ReLU())
    def forward(self, x):
        x=self.Conv (x)
        x1 = self.RCNN3(x)
        x2 = self.RCNN2(x)
        x3 = self.RCNN1(x)
        out = self.Conv1_1(x3+x2+x1)
        return out
class Attention_block_self(nn.Module):
    """
    Attention Block
    """

    def __init__(self, F_l, F_int):
        super(Attention_block_self, self).__init__()

        self.W_g = nn.Sequential(
            nn.Conv1d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm1d(F_int)
        )

        self.psi = nn.Sequential(
            nn.Conv1d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm1d(1),
            nn.Sigmoid()
        )

        self.Tanh = nn.Tanh()

    def forward(self,  x):
        x1 = self.W_g(x)
        psi = self.Tanh(x1)
        psi = self.psi(psi)
        out = x * psi
        return out


class model_1d(nn.Module):
    """
    R2U-Unet implementation
    Paper: https://arxiv.org/abs/1802.06955
    """
    def __init__(self, img_ch=1, output_ch=1, t=1):
        super(model_1d, self).__init__()

        n1 =6
        filters = [n1, n1 * 2, n1 * 4, n1 * 8, n1 * 16]
        self.Maxpool0 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.Maxpool1 = nn.MaxPool1d(kernel_size=8, stride=8)
        self.Maxpool2 = nn.MaxPool1d(kernel_size=4, stride=4)

        self.RRCNN1 = R_inception_RCNN_block(img_ch, filters[3], t=t)

        self.RRCNN2 = R_inception_RCNN_block(filters[3], filters[2], t=t)

        #self.RRCNN3 = R_inception_RCNN_block(filters[2], filters[2], t=t)
        self.RRCNN3 = R_inception_RCNN_block(filters[2], filters[2], t=t)
        #self.RRCNN4 = RRCNN_block(filters[3], filters[1], t=t)
        # self.Attention_block=Attention_block_self( F_l=filters[2], F_int=2)
        self.Softmax = nn.LogSoftmax()
        self.fc1 = nn.Linear(168,2)
        #self.fc2 = nn.Linear(3 ,3)
        #self.att= LinearSelfAttn()
        #self.fc2 = nn.Linear(24,1)
    def forward(self, x):
        x=self.Maxpool0(x)
        e1 = self.RRCNN1(x)
        #e1 = self.drop_layer(e1)

        e2 = self.Maxpool2(e1)

        e2 = self.RRCNN2(e2)
        #e2 = self.drop_layer(e2)

        e3 = self.Maxpool2(e2)
        e3 = self.RRCNN3(e3)
        e3 = self.Maxpool2(e3)
        '''
        #e3 = self.drop_layer(e3)

        e4 = self.Maxpool2(e3)
        e4 = self.RRCNN3(e4)

        e4= self.Maxpool2(e3)
        '''
        e4=self.Maxpool2(e3)
        #e4 = self.Attention_block(x=e4)
        # print(e4.size(1))
        # print(e4.size(2))
        e7= e4.view(e4.size(0), e4.size(1)*e4.size(2))
        #e7=e4.view(-1, e4.view(-1).size(0))
        out = self.fc1(e7)
        #out = self.fc2(out)
        out =self.Softmax(out)
        #out= out.view(-1,24)
        #out=self.fc2(out)
        return out


def test(model,testloader):
     model.eval()
     test_loss = 0.0
     test_correct=0.0
     labelpredict=[]
     label=[]
     data=[]
     for inputs1, labels1 in testloader:
        inputs1, labels1 = Variable(inputs1.cuda()), Variable(labels1.cuda())
        # print(torch.cuda.is_initialized())
        output = model(inputs1)

        data.append(inputs1.cpu().numpy())
        label.append(torch.argmax(labels1.cpu(),1).numpy())
        labelpredict.append(torch.argmax(output.cpu(),1))
     test_correct/=(len(testloader.dataset))
     return labelpredict ,data


# data=sio.loadmat('ecgpart_04.mat')
with open('testing_dataset.pkl', 'rb') as f:
     data = pickle.load(f)
# data = main_dataset.iloc[test_index].values
ecga=data[:, :4000]
print(len(ecga))
#useless-------------------------------------------------------------
# label1=np.zeros((len(ecga)-500,1))
# label2=np.zeros((250,1))+1
# label3=np.zeros((250,1))+2
# labelt=np.vstack((label1,label2,label3))
#---------------------------------------------------------------------
# label=sio.loadmat('CSPC2020label_04_0_1_by_manurally.mat')['sqi']
labelt = data[:, -1]
labelt = labelt.reshape(198, 1)
# labelt = label['sqi'][0:500]
print(labelt.shape)
for FF1 in range(len(ecga)):
    ecga[FF1,:]=butthigh(zscore(ecga[FF1,:]),400)
ecgt=torch.FloatTensor(ecga)
ecgt=ecgt.unsqueeze(1)

labelt=to_categorical(labelt)
labelt=torch.FloatTensor(labelt)
print(labelt.size)
deal_test_dataset = TensorDataset(ecgt,labelt)

testloader=DataLoader(dataset=deal_test_dataset,batch_size=32,shuffle=False,num_workers=0)
modelname='conmodel_4_epoch199.pkl'
model=torch.load(modelname)
# print(model)
model.eval()
labelpredict,testdata11=test(model,testloader)
j1=[]
for j in labelpredict:
    j2=j.numpy()
    j1.extend(j2)
# matname_result='2020label_04_result_trained_on_computer.mat'
# sio.savemat(matname_result,{"predict":j1})
filename = '2011_label_predict_by_model_trainied_on_patient_02.pkl'
import pickle
with open(filename, 'wb') as f:
  pickle.dump(j1, f)



198
(198, 1)
<built-in method size of Tensor object at 0x785127850b80>


<ipython-input-19-34a3adbe3c7e>:305: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out =self.Softmax(out)


In [13]:
with open("testing_label_2020_patient_02.pkl", 'wb') as f:
  pickle.dump(data[:, -1], f)

In [14]:
with open('testing_label_2020_patient_02.pkl', 'rb') as f:
    actual = pickle.load(f)

In [15]:
actual.shape

(1644,)

In [17]:
with open('2020_label_predict_patient_02.pkl', 'rb') as f:
    predicted = pickle.load(f)

In [18]:
np.array(predicted).shape

(1644,)

In [10]:
train_index.shape

(6574,)

In [11]:
test_index.shape

(1644,)